# Building a Multi-Agent Deep Researcher

## Setup

### Install Packages

Resolve this issue in CrewAI before proceeding: https://github.com/crewAIInc/crewAI/issues/2606

### API Keys

We'll need the Linkup API key for building our Deep Researcher, you can sign up [here](https://app.linkup.so/sign-up).

In [1]:
import os, json, re, getpass
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv(override=True)

True

In [2]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [3]:
#Check for OpenAI API Key
_set_env("OPENAI_API_KEY")

### Libraries

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from typing import Type
from pydantic import BaseModel, Field
from linkup import LinkupClient
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool

## Selecting Components

### Chat Model

Check [this](https://docs.crewai.com/en/concepts/llms) for available LLM integrations.

In [6]:
llm = LLM(
    # model="groq/llama3-8b-8192",
    model="openai/gpt-4o-mini",
    # model ="groq/llama-3.2-90b-text-preview"
    # temperature=0.5,
    # max_completion_tokens=1024,
    # top_p=0.9,
    # stop=None,
    # stream=False,
)

In [7]:
llm.call("hello")

'Hello! How can I assist you today?'

### Define Linkup Search Tool

In [8]:
class LinkupSearchInput(BaseModel):
    """Input schema for Linkup Search Tool."""
    query: str = Field(description="The search query to perform")
    depth: str = Field(default="standard",
                       description="Depth of search: 'standard' or 'deep'")
    output_type: str = Field(
        default="searchResults", description="Output type: 'searchResults', 'sourcedAnswer', or 'structured'")

In [9]:
class LinkupSearchTool(BaseTool):
    name: str = "Linkup Search"
    description: str = "Search the web for information using Linkup and return comprehensive results"
    args_schema: Type[BaseModel] = LinkupSearchInput

    def __init__(self):
        super().__init__()

    def _run(self, query: str, depth: str = "standard", output_type: str = "searchResults") -> str:
        """Execute Linkup search and return results."""
        try:
            # Initialize Linkup client with API key from environment variables
            linkup_client = LinkupClient(api_key=os.getenv("LINKUP_API_KEY"))

            # Perform search
            search_response = linkup_client.search(
                query=query,
                depth=depth,
                output_type=output_type
            )

            return str(search_response)
        except Exception as e:
            return f"Error occurred while searching: {str(e)}"

In [10]:
# Initialize tools
linkup_search_tool = LinkupSearchTool()

## Setting up Crew of Agents

In CrewAI, you define **agents** as independent workers, each with a specific job and objective. You then assign them **tasks**, which are detailed instructions on what they need to do to reach that objective. You can also assign **tools** directly as tasks for agents to use.

**Let's first create our CrewAI agents with role, main goal/objective, and backstory/personality.**

### Web Searcher Agent

In [11]:
web_searcher = Agent(
        role="Web Searcher",
        goal="Find the most relevant information on the web, along with source links (urls) for this user query: {query}.",
        backstory="An expert at formulating search queries and retrieving relevant information. Passes the results to the 'Research Analyst' only.",
        verbose=True,
        allow_delegation=True,
        tools=[linkup_search_tool],
        llm=llm,
    )

### Research Analyst Agent

In [12]:
research_analyst = Agent(
        role="Research Analyst",
        goal="Analyze and synthesize raw information into structured insights, along with source links (urls) as citations.",
        backstory="An expert at analyzing information, identifying patterns, and extracting key insights. If required, can delagate the task of fact checking/verification to 'Web Searcher' only. Passes the final results to the 'Technical Writer' only.",
        verbose=True,
        allow_delegation=True,
        llm=llm,
    )

### Technical Writer Agent

In [13]:
technical_writer = Agent(
        role="Technical Writer",
        goal="Create well-structured, clear, and comprehensive responses in markdown format, with citations/source links (urls).",
        backstory="An expert at communicating complex information in an accessible way.",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

**Now, let's define the agents' tasks.**

### Web Searcher Agent - Task

In [14]:
search_task = Task(
        description="Search for comprehensive information about the user query: {query}.",
        agent=web_searcher,
        expected_output="Detailed raw search results including sources (urls).",
        tools=[linkup_search_tool]
    )

### Research Analyst Agent - Task

In [15]:
analysis_task = Task(
        description="Analyze the raw search results, identify key information, verify facts and prepare a structured analysis.",
        agent=research_analyst,
        expected_output="A structured analysis of the information with verified facts and key insights, along with source links",
        context=[search_task]
    )

### Technical Writer Agent - Task

In [16]:
writing_task = Task(
        description="Create a comprehensive, well-organized response based on the research analysis.",
        agent=technical_writer,
        expected_output="A clear, comprehensive response that directly answers the query with proper citations/source links (urls).",
        context=[analysis_task]
    )

**Now, let's create crew to manage agents and task workflow.**

In [17]:
crew = Crew(
        agents=[web_searcher, research_analyst, technical_writer],
        tasks=[search_task, analysis_task, writing_task],
        verbose=True,
        process=Process.sequential
    )

In [18]:
# Run the crew with an example query
result = crew.kickoff(inputs={"query": "What are the best books from industry writers to learn Generative AI?"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3019ef97-4581-45b7-b2a8-4c4b13436903                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Task: Search for comprehensive information about the user query: What are the best books from industry         │
│  writers to learn Generative AI?.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Thought: Action: Linkup Search                                                                                 │
│                                                                                                                 │
│  Using Tool: Linkup Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"best books from industry writers to learn Generative AI\", \"depth\": \"deep\",                 │
│  \"output_type\": \"searchResults\"}"                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Analyst                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I will reevaluate the descriptions to ensure they are both direct and concise. I should      │
│  avoid unnecessary complexity and focus purely on the essentials to eliminate validation issues.                │
│                                                                                                                 │
│  Using Tool: Delegate work to coworker                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"task\": {\"description\": \"Gather key information on books about Generative AI (titles, authors, themes)  │
│  from these sources.\", \"type\": \"str\"}, \"context\": {\"description\": \"Sources: 1. Medium article about   │
│  mastering Generative AI. 2. Analytics Vidhya's 10 GenAI books. 3. BookAuthority's expert-level books. 4.       │
│  MentorCruise's top recommendations. 5. Business Insider's best books for learning AI. 6. Innovation            │
│  Training's top Generative AI books. 7. Medium's professional growth reading. 8. GetGuru's guide to Generative  │
│  AI. 9. R-bloggers' reviews of best AI books. 10. Manning's introduction to Generative AI.\", \"type\":         │
│  \"str\"}, \"coworker\": \"Web Searcher\"}"                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2        │
│  validation errors for DelegateWorkToolSchema                                                                   │
│  task                                                                                                           │
│    Input should be a valid string [type=string_type, input_value={'description': 'Gather k...ources.', 'type':  │
│  'str'}, input_type=dict]                                                                                       │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type                               │
│  context                                                                                                        │
│    Input should be a valid string [type=string_type, input_value={'description': "Sources:...ive AI.", 'type':  │
│  'str'}, input_type=dict]                                                                                       │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type.                              │
│   Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker                     │
│  Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description':    │
│  'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the coworker to      │
│  delegate to', 'type': 'str'}}                                                                                  │
│  Tool Description: Delegate a specific task to one of the following coworkers: Web Searcher, Technical Writer   │
│  The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to      │
│  execute the task, they know nothing about the task, so share absolutely everything you know, don't reference   │
│  things but instead explain them..                                                                              │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Delegate work to coworker, Ask question to coworker]             │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer               

Output()

KeyboardInterrupt: 

In [ ]:
#Print result
display(Markdown(result.raw))